In [1]:
!python3 -m grpc_tools.protoc -I /opt/proto --python_out=. --pyi_out=. --grpc_python_out=. /opt/proto/game.proto

In [ ]:
!python3 -m pip install numpy==1.23.5

In [3]:
import game_pb2_grpc
import game_pb2
import grpc
from subprocess import Popen, PIPE
import time
import traceback

In [4]:
def start_server(port, seed):
    p = Popen(['xvfb-run', '--server-num' , str(port), r'./artifical_bike_racing', '--port', str(port), '--seed', str(seed), '--headless', '--save', f'/opt/clients/{port}.json' ], stdout=PIPE, cwd=r'/opt/')
    broken = True
    while broken:
        try:
            channel = grpc.insecure_channel(f'localhost:{port}')
            stub = game_pb2_grpc.MainServiceStub(channel)
            stub.health(game_pb2.Empty())
            broken = False
        except:
            pass
        
    return stub, p.stdout

In [5]:
def kill_server(stub):
    stub.kill(game_pb2.Empty())
    try:
        stub.input(game_pb2.InputRequest(x = 0.0, z = 1.0))
        stub.getState(game_pb2.Empty())
        broken = True
        while broken:
            try:
                stub.health(game_pb2.Empty())
            except:
                broken = False
    except:
        pass

In [6]:
def get_score(stub):
    score = stub.getScore(game_pb2.Empty())
    try:
        stub.input(game_pb2.InputRequest(x = 0.0, z = 1.0))
        stub.getState(game_pb2.Empty())
        broken = True
        while broken:
            try:
                stub.health(game_pb2.Empty())
            except:
                broken = False
    except:
        pass
    return score

In [ ]:
# retrieve the terrain at a given offset from the player, x = 0, z = 0 returns the terrain below the player
def terrain_at(terrain, x, z):
    if x < -32 or x > 32 or z < -32 or z > 32:
        return None
    x = x + 32
    z = z + 32
    terrain[x + z * 65]

In [ ]:
# runs the game for {steps} frames for {seed}
# the decision_f gets the frame state as an input and should output the x and z velocity
# the recording is stored under /opt/clients/{filename}.json and can be replayed as described in the readme
def play_game(port, seed, filename, decision_f, steps):
    stub, process = start_server(port, seed)
    stub.input(game_pb2.InputRequest(x = 0.0, z = 1.0))
    state = stub.getState(game_pb2.Empty())
    for _ in range(steps):
        d = []
        if state.finished:
            return stub, state
        if state.y < -50:
            return stub, state
        x, z = decision_f(state)
        
        stub.input(game_pb2.InputRequest(x = x, z = z))
        state = stub.getState(game_pb2.Empty())
    return stub, state